In [190]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pypfopt import EfficientFrontier
import itertools

In [191]:
file_path = r"C:\Users\kamod\OneDrive\Escritorio\2024-2 U\Intc Soc\modelos-de-interacciones-sociales--main\trabajo final\Datos\Price_Stocks.xlsx"

retorno_df = pd.read_excel(file_path, sheet_name="Retorno")

In [192]:
n_activos = 15
periodos=200 # este es T
n_portafolios = 10

In [193]:
retorno_df_clean = retorno_df.dropna()
retorno_df_clean.loc[:, 'Exchange Date'] = pd.to_datetime(retorno_df_clean['Exchange Date'], errors='coerce')
retorno_df_clean = retorno_df_clean.set_index('Exchange Date')
retorno_df_clean = retorno_df_clean.select_dtypes(include=['float64', 'int64'])
#retorno_df_clean = retorno_df_clean*100
# Sort data by Exchange Date in ascending order
retorno_df_clean = retorno_df_clean.sort_index()
df = retorno_df_clean.iloc[:, :n_activos]
fechas = df.index

In [194]:
df

,AAP,AAPL,AXP,AZO,CPB,CSCO,CVX,DUK,DXC,EMR,EQT,FDX,GIS,GS,HAL
Exchange Date,,,,,,,,,,,,,,,
2010-01-05,-0.005944,0.001729,-0.002199,-0.008353,0.001771,-0.004455,0.007083,-0.016500,-0.000346,0.001615,0.006921,0.013062,-0.009309,0.017680,0.012800
2010-01-06,0.008719,-0.015906,0.016165,-0.009380,-0.011200,-0.006509,0.000126,0.003595,0.003638,0.008982,0.003991,-0.008280,0.005979,-0.010673,0.023697
2010-01-07,-0.000247,-0.001849,0.011810,0.013270,-0.015201,0.004505,-0.003767,0.004776,-0.010012,0.000000,-0.013472,-0.010854,0.009057,0.019568,0.002469
2010-01-08,0.003953,0.006648,-0.000715,-0.012842,-0.006356,0.005300,0.001765,0.000594,-0.002616,0.005706,-0.008507,0.024840,-0.008135,-0.018911,0.050493
2010-01-11,-0.009843,-0.008822,-0.011442,0.001288,0.003046,-0.002839,0.017743,0.010095,-0.006119,0.015207,-0.000903,0.026591,0.000990,-0.015776,-0.009965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-23,-0.023226,-0.021623,-0.004295,0.007724,-0.008446,-0.005654,-0.002915,0.005756,-0.019062,-0.008860,0.006116,-0.006449,-0.007238,-0.002122,-0.015227
2024-10-24,-0.015059,-0.000823,-0.004609,-0.013546,0.000639,-0.005686,-0.000199,-0.012856,-0.006477,-0.002304,0.021277,0.017982,0.000437,0.014114,0.001438
2024-10-25,-0.014485,0.003643,-0.009741,-0.007069,0.001702,-0.003931,0.002393,-0.015208,-0.003511,-0.001201,0.014069,-0.002749,-0.004518,-0.022688,0.014363


In [195]:
  # Número de portafolios
pesos = np.random.rand(n_portafolios, n_activos)  # 48 portafolios, 5 activos
pesos = pesos / pesos.sum(axis=1)[:, np.newaxis]  # Normalizamos para que sumen 1

# Multiplicamos los retornos por los pesos para obtener los portafolios
portafolios = np.dot(df.values, pesos.T)

# Paso 3: Crear un DataFrame para los portafolios con los valores calculados
df_portafolios = pd.DataFrame(portafolios, index=df.index, columns=[f'Portafolio_{i+1}' for i in range(n_portafolios)])
df_portafolios

,Portafolio_1,Portafolio_2,Portafolio_3,Portafolio_4,Portafolio_5,Portafolio_6,Portafolio_7,Portafolio_8,Portafolio_9,Portafolio_10
Exchange Date,,,,,,,,,,
2010-01-05,-0.000845,0.001864,-0.000620,0.000907,0.001941,-0.001617,0.000192,-0.000473,0.002558,-0.001226
2010-01-06,0.001905,0.000209,0.003004,0.001568,-0.000515,0.002817,-0.000562,0.001668,-0.000632,0.003055
2010-01-07,0.004655,0.000153,-0.000206,0.002005,-0.000054,0.002373,0.001104,-0.000008,0.001203,-0.000208
2010-01-08,0.002128,0.004147,0.005723,0.004720,0.001880,0.005826,0.000588,0.006933,0.002758,0.003228
2010-01-11,-0.001469,-0.000321,0.000350,-0.001076,0.000162,0.001535,-0.001014,0.002531,0.000704,0.000902
...,...,...,...,...,...,...,...,...,...,...
2024-10-23,-0.005871,-0.008642,-0.008546,-0.008645,-0.006973,-0.006960,-0.007377,-0.008357,-0.006607,-0.005758
2024-10-24,-0.002857,0.000766,-0.002635,-0.002416,0.001985,-0.002624,-0.001588,-0.001919,0.001689,-0.001801
2024-10-25,-0.006140,-0.002020,-0.003846,-0.003936,-0.003023,-0.002155,-0.004076,-0.003546,-0.001669,-0.002503


In [196]:
df_portafolios

,Portafolio_1,Portafolio_2,Portafolio_3,Portafolio_4,Portafolio_5,Portafolio_6,Portafolio_7,Portafolio_8,Portafolio_9,Portafolio_10
Exchange Date,,,,,,,,,,
2010-01-05,-0.000845,0.001864,-0.000620,0.000907,0.001941,-0.001617,0.000192,-0.000473,0.002558,-0.001226
2010-01-06,0.001905,0.000209,0.003004,0.001568,-0.000515,0.002817,-0.000562,0.001668,-0.000632,0.003055
2010-01-07,0.004655,0.000153,-0.000206,0.002005,-0.000054,0.002373,0.001104,-0.000008,0.001203,-0.000208
2010-01-08,0.002128,0.004147,0.005723,0.004720,0.001880,0.005826,0.000588,0.006933,0.002758,0.003228
2010-01-11,-0.001469,-0.000321,0.000350,-0.001076,0.000162,0.001535,-0.001014,0.002531,0.000704,0.000902
...,...,...,...,...,...,...,...,...,...,...
2024-10-23,-0.005871,-0.008642,-0.008546,-0.008645,-0.006973,-0.006960,-0.007377,-0.008357,-0.006607,-0.005758
2024-10-24,-0.002857,0.000766,-0.002635,-0.002416,0.001985,-0.002624,-0.001588,-0.001919,0.001689,-0.001801
2024-10-25,-0.006140,-0.002020,-0.003846,-0.003936,-0.003023,-0.002155,-0.004076,-0.003546,-0.001669,-0.002503


In [197]:
# Paso 4: Calcular la matriz de covarianzas con una ventana móvil
ventana = 150  # Tamaño de la ventana móvil (ajustado para que funcione con los 20 días)
cov_matrices = []
for i in range(ventana, len(df_portafolios)):
    ventana_datos = df_portafolios.iloc[i-ventana:i]
    cov_matrices.append(ventana_datos.cov().values)


In [198]:
# Lista para almacenar los portafolios dinámicos
portafolios_dinamicos = []

for t, cov in enumerate(cov_matrices):
    cov_matrix = cov

    # Paso 5: Descomponer la matriz de covarianzas (autovalores y autovectores)
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

    # Paso 6: Ordenar los autovalores y autovectores en orden descendente
    idx = eigenvalues.argsort()[::-1]  # Índices ordenados
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    eigenvectors = eigenvectors.astype(float)

    eigenvectors_normalized = []
    # Paso 7: Normalizar los eigenvectores seleccionados
    for i in range(eigenvectors.shape[1]):
        col_sum = np.sum(np.abs(eigenvectors[:, i]))
        eigenvectors[:, i] = np.abs(eigenvectors[:, i] / col_sum)

    # Paso 7: Normalizar los eigenvectores seleccionados
    eigenvectors_normalized = eigenvectors

    # Paso 8: Crear portafolios ortogonales a partir de los eigenvectores normalizados
    df_ortogonales = pd.DataFrame(
        np.dot(df_portafolios.values, eigenvectors_normalized),
        index=fechas,
        columns=[f'portafolio_ortogonal_{i+1}' for i in range(eigenvectors_normalized.shape[1])]
    )
    
    # Paso 9: Determinar dinámicamente el número de eigenportafolios principales (l)
    mediana_eigenvalores = np.median(eigenvalues)  # Calcular la mediana de los eigenvalores
    l = sum(eigenvalues > mediana_eigenvalores)  # Contar cuántos eigenvalores superan la mediana

    # Paso 10: Seleccionar los primeros l portafolios ortogonales
    df_ortogonales_l = df_ortogonales.iloc[:, :l]
    portafolios_dinamicos.append(df_ortogonales_l)

    # Mostrar los pesos de los portafolios principales
    #print(f"Número de eigenportafolios seleccionados dinámicamente l={l} en t={t}")
print(f"Pesos de los portafolios principales (autovectores normalizados):")
print(eigenvectors_normalized[:, :l])  # Los pesos de los primeros 'l' portafolios ortogonales

# Mostrar los primeros l portafolios ortogonales
print(df_ortogonales_l.head())

Pesos de los portafolios principales (autovectores normalizados):
[[0.10361108 0.04101254 0.09888676 0.05588922 0.13627694]
 [0.09589918 0.03846655 0.09037841 0.07555055 0.1023701 ]
 [0.10826995 0.09618548 0.05553192 0.07760536 0.12781652]
 [0.10798164 0.03798434 0.06649882 0.1125329  0.04030076]
 [0.10274921 0.11900397 0.06277439 0.16942382 0.05886862]
 [0.08719789 0.16373179 0.08386964 0.11915124 0.1306767 ]
 [0.10673064 0.10311163 0.24878197 0.14784531 0.00825356]
 [0.1011517  0.22312633 0.06803765 0.04009005 0.09542563]
 [0.09089085 0.11993611 0.18058917 0.10537388 0.01918793]
 [0.09551788 0.05744128 0.04465127 0.09653767 0.28082324]]
               portafolio_ortogonal_1  portafolio_ortogonal_2  \
Exchange Date                                                   
2010-01-05                   0.000269                0.000129   
2010-01-06                   0.001248                0.001248   
2010-01-07                   0.001099                0.000880   
2010-01-08                  

In [199]:
np.sum(eigenvectors_normalized, axis=0, keepdims=True)

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [200]:
filas = []

# Supongamos que portafolios_dinamicos es una lista de DataFrames
for i in range(len(portafolios_dinamicos)):
    # Tomamos la fila correspondiente, incrementando el índice con el valor de 'i'
    fila = portafolios_dinamicos[i].iloc[ventana-1 + i]  # Ajustamos el índice de la fila
    filas.append(fila)

# Convertir la lista de filas en un DataFrame
resultado_df = pd.DataFrame(filas)

# Mostrar el DataFrame resultante
resultado_df

,portafolio_ortogonal_1,portafolio_ortogonal_2,portafolio_ortogonal_3,portafolio_ortogonal_4,portafolio_ortogonal_5
2010-08-09,0.007113,0.007102,0.007174,0.007227,0.007081
2010-08-10,-0.003082,-0.002909,-0.002949,-0.003426,-0.003051
2010-08-11,-0.029037,-0.029041,-0.029334,-0.029054,-0.029144
2010-08-12,-0.008635,-0.007163,-0.008759,-0.009385,-0.008636
2010-08-13,-0.003528,-0.003486,-0.003560,-0.003572,-0.003547
...,...,...,...,...,...
2024-10-22,-0.005405,-0.005328,-0.005112,-0.005115,-0.005027
2024-10-23,-0.007402,-0.007447,-0.007262,-0.007341,-0.007054
2024-10-24,-0.001175,-0.001132,-0.000910,-0.000886,-0.001609
2024-10-25,-0.003354,-0.003120,-0.003306,-0.003150,-0.003268


In [201]:
resultado_df.mean()

portafolio_ortogonal_1    0.000441
portafolio_ortogonal_2    0.000442
portafolio_ortogonal_3    0.000440
portafolio_ortogonal_4    0.000445
portafolio_ortogonal_5    0.000438
dtype: float64

In [202]:
resultado_df = df

In [203]:
import numpy as np
from scipy.stats import beta
import pandas as pd

def adaptive_discounted_thompson_sampling(returns, gamma, w):
    """
    Adaptive Discounted Thompson Sampling (Algorithm 1).

    Parameters:
        returns (pd.DataFrame): DataFrame donde las filas son días y las columnas son portafolios,
                                y los valores son los retornos diarios.
        gamma (float): Factor de descuento (0 < gamma <= 1).
        w (int): Tamaño de la ventana deslizante (1 <= w <= T).

    Returns:
        actions (list): Índices de los portafolios seleccionados cada día.
        observed_rewards (list): Retornos observados para los portafolios seleccionados.
    """
    T, K = returns.shape  # T: número de días, K: número de portafolios
    returns_binario=  returns.apply(lambda row: row == row.max(), axis=1).astype(int)
    # Inicializar parámetros alpha y beta
    alpha = np.ones(K)  # α_k inicializado en 1
    beta_params = np.ones(K)  # β_k inicializado en 1

    actions = []  # Almacenar los portafolios elegidos
    observed_rewards = []  # Almacenar los retornos observados
    eventos=[]

    for t in range(T):
        # Step 2: Calcular θ_k(t) y θ̌_k(t) para cada portafolio
        theta = []
        theta_tilde = []
        for k in range(K):
            # θ_k(t) ~ Beta(α_k + 1, β_k + 1)
            theta_k = beta.rvs(alpha[k] + 1, beta_params[k] + 1)
            theta.append(theta_k)
            
            # θ̌_k(t) ~ Beta(α_k^w, β_k^w), utilizando las últimas w recompensas
            if t < w:
                ventana_reward_t=returns_binario.iloc[0:t-1, k]  # Retorna lo mas que pueda
            else:
                ventana_reward_t=returns_binario.iloc[t-w-1:t-1, k]  # Si t es mayor que w, retorna los datos t-w
        
            alpha_w = np.count_nonzero (ventana_reward_t.values)  # Suponiendo recompensas positivas son éxitos
            beta_w = len(ventana_reward_t.values)- alpha_w
            theta_tilde_k = beta.rvs(alpha_w + 1, beta_w + 1)
            theta_tilde.append(theta_tilde_k)
            
        
        # Step 6: Elegir el portafolio I(t) = argmax_k f(θ_k(t), θ̌_k(t))
        f_values = [(theta[k] + theta_tilde[k])/2 for k in range(K)]
        I_t = np.argmax(f_values)  # Índice del portafolio seleccionado con la ponderacion
        actions.append(I_t)

        # Step 7: Observar el retorno r_t del portafolio seleccionado
        reward_t = returns.iloc[t, I_t]
        observed_rewards.append(reward_t)

        # Step 8: Calcular X_t (1 si retorno es positivo, 0 en caso contrario)
        X_t = 1 if reward_t == returns.iloc[t, :].max() else 0
        eventos.append(X_t)
        

        # Step 9: Actualizar Beta(α_k, β_k) para todos los portafolios
        for k in range(K): #acá actualizó tanto el que disparo como el máximo
            if k == I_t:
                alpha[k] = gamma * alpha[k] + 1
                beta_params[k] = gamma * beta_params[k] 
            else:
                alpha[k] =  gamma * alpha[k] 
                beta_params[k] = gamma * beta_params[k] + 1
        
    return actions, observed_rewards, eventos


# Simular DataFrame con tus retornos (cargar tus datos reales aquí)
returns = resultado_df

# Parámetros del algoritmo
gamma = 0.8  # Factor de descuento
w = 90  # Tamaño de la ventana deslizante

# Ejecutar el algoritmo
actions, observed_rewards, eventos = adaptive_discounted_thompson_sampling(returns, gamma, w)

# Resultados
print("Portafolios seleccionados cada día:", actions)
#print("Retornos observados:", observed_rewards)
print("eventos:", eventos)
numero_de_unos = eventos.count(1)
print(" numero aciertos:", numero_de_unos)

Portafolios seleccionados cada día: [np.int64(0), np.int64(1), np.int64(5), np.int64(5), np.int64(10), np.int64(13), np.int64(10), np.int64(11), np.int64(14), np.int64(11), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), np.int64(14), n

In [204]:
observed_rewards_df = pd.DataFrame(observed_rewards)

# Start with initial wealth of 1 dollar
initial_wealth = 1

# Create an empty DataFrame to hold the wealth over time for each portfolio
wealth = pd.DataFrame(index=observed_rewards_df.index, columns=observed_rewards_df.columns)

# Iterate over each portfolio (column)
for portfolio in observed_rewards_df.columns:
    # Compute cumulative wealth for each portfolio
    wealth[portfolio] = (1 + observed_rewards_df[portfolio]).cumprod() * initial_wealth

In [205]:
observed_rewards

[np.float64(-0.00594353640416052),
 np.float64(-0.0159063875928453),
 np.float64(0.00450450450450448),
 np.float64(0.00529963310232364),
 np.float64(-0.000903110207360276),
 np.float64(-0.0217999533690838),
 np.float64(0.00732770851041048),
 np.float64(0.0054765788860407),
 np.float64(-0.0081608860390557),
 np.float64(0.00928210551051588),
 np.float64(-0.0384393063583814),
 np.float64(-0.0222422602945597),
 np.float64(-0.0424223793421458),
 np.float64(-0.00256821829855532),
 np.float64(-0.00611522368844549),
 np.float64(-0.015220207253886),
 np.float64(-0.0249917790200592),
 np.float64(-0.014839797639123),
 np.float64(0.0492981855528928),
 np.float64(0.00358890701468199),
 np.float64(-0.0130039011703512),
 np.float64(-0.0494071146245059),
 np.float64(-0.0197505197505198),
 np.float64(-0.00671615411806284),
 np.float64(0.0348754448398575),
 np.float64(0.00515818431911974),
 np.float64(0.0318166267533356),
 np.float64(-0.00464190981432363),
 np.float64(0.0343104596935377),
 np.float64(0.

In [206]:
wealth

,0
0,0.994056
1,0.978245
2,0.982651
3,0.987859
4,0.986967
...,...
3725,2.875986
3726,2.857357
3727,2.847326
3728,2.823828
